In [1]:
## Generate a graph
using StatsBase
using Combinatorics

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/hypergraph_louvain.jl")
include("jl/inference.jl")

In [16]:
n = 100
Z = rand(1:2, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 3

Ω = buildΩ(ω, α0, kmax)
## Sample
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((100,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
  E: Dict{Int64,Dict}
  D: Array{Int64}((100,)) [4, 1, 0, 1, 1, 0, 2, 2, 0, 4  …  0, 0, 3, 1, 1, 1, 1, 1, 2, 0]


In [110]:
## Run Louvains, Naive, and Faster version
include("jl/hypergraph_louvain.jl")
@time Zlou2 = HyperLouvain(H,kmax,Ω;α=α0)
@time Zlou = Naive_HyperLouvain(H,Ω;α=α0)

mlou = modularity(H,Zlou,Ω,α=α0)
mlou2 = modularity(H,Zlou2,Ω,α=α0)

# This may not be exactly the same, but that's okay.
# Even a difference of 1e-14 in modularity change may lead you to make a 
# different greedy decision, but it's still a valid move.
@show mlou, mlou2
@show abs(mlou- mlou2)


Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
tdiff = 0
  1.644867 seconds (1.89 M allocations: 94.755 MiB, 4.45% gc time)

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
  1.854632 seconds (5.07 M allocations: 979.449 MiB, 23.50% gc time)
mlou ≈ mlou2 = false
abs(mlou - mlou2) = 0.33520787515331142764868463564120748543473382596857845783233642578125


0.33520787515331142764868463564120748543473382596857845783233642578125

In [107]:
## Try supernode Louvain
# Sanity check: having a singletons initialization is equivalent to naive louvain
include("jl/hypergraph_louvain.jl")
Z = collect(1:n)
@time Znew, changed = Naive_SuperNodeStep(H,Z,kmax,Ω;α=α0)
mlou3 =  modularity(H,Znew,Ω,α=α0)


Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
  2.181428 seconds (6.59 M allocations: 1.129 GiB, 19.03% gc time)


-893.4706412410682598171788909753587848427969220210798084735870361328125

In [121]:
## Now repeatedly call the Naive_SuperNodeStep until no more improvement is possible
include("jl/hypergraph_louvain.jl")
@time Z = SuperNodeLouvain(H,kmax,Ω;α=α0)
msuper = modularity(H,Z,Ω,α=α0)

SuperNode Louvain: Phase 1

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
SuperNode Louvain: Phase 2

Louvain Iteration 1
(abs(change3 - change), abs(change2 - change)) = (2.753353101070388220250606536865234375e-14, 0.0)
(abs(change3 - change), abs(change2 - change)) = (1.7763568394002504646778106689453125e-15, 0.0)
(abs(change3 - change), abs(change2 - change)) = (2.753353101070388220250606536865234375e-14, 0.0)
(abs(change3 - change), abs(change2 - change)) = (0.135155036036056941384231322444975376129150390625, 0.0)


AssertionError: AssertionError: abs(change - change3) < 1.0e-10